In [ ]:
import numpy as np
import pandas as pd
import geneselection.solvers.elasticnet.linear as elin
import geneselection.solvers.elasticnet.utils as eutils
import geneselection.datasets.scrnaseq_cardio_20181129 as cardio

In [ ]:
params = dict(days = "all",                                    # list of days to use (or "all")
              X_noise = 0.01,                                  # gaussian noise added to scaled X
              y_noise = 0.5,                                   # gaussian noise added to scaled y
              day_offset = 0.1,                                # offest for close but similar days eg D24 vs D26
              lambda_path = np.geomspace(1.0, 0.01, num=100),  # lambda path
              alpha = 0.9,                                     # fraction of regularization devoted to L1 prenalty
              n_bootstraps = 100,                              # number of bootstrap replicates
              n_processes = 25,                                # number of parallel processes to use
              thresholds = np.linspace(0.01,1,num=100))        # selection thresholds for including genes

In [ ]:
adata = cardio.load()
adata = eutils.preprocess_cardio(adata)
adata = eutils.subset_cardio(adata, days=params["days"])

In [ ]:
boot_results = elin.parallel_runs(adata,
                                  n_processes=params["n_processes"],
                                  n_bootstraps=params["n_bootstraps"],
                                  X_noise=params["X_noise"],
                                  y_noise=params["y_noise"],
                                  alpha=params["alpha"],
                                  lambda_path=params["lambda_path"],
                                  day_offset=params["day_offset"])

In [ ]:
eutils.thresh_lambda_plot(boot_results,
                          adata,
                          thresholds=params["thresholds"],
                          lambdas=params["lambda_path"])

In [ ]:
predictive_genes = eutils.get_selected_genes(boot_results,
                                             adata,
                                             lambda_index=65,
                                             selection_threshold_index=90,
                                             thresholds=params["thresholds"])

In [ ]:
predictive_genes